In [23]:
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.models import load_model
import pandas as pd

In [24]:
# Load the trained model 
model = load_model('model.h5')

# Load encoder and scaler objects
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [25]:
#Example input data
input_data = { 
    "CreditScore": 619,
    "Geography": "France",
    "Gender": "Female",
    "Age": 42,
    "Tenure": 2,
    "Balance": 0.00,
    "NumOfProducts": 1,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 101348.88,
    } 
    

In [26]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Lenovo\Documents\udemy_ann\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
input_df = pd.DataFrame([input_data])


In [28]:
# Encode categorical values 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.0,1,1,1,101348.88


In [29]:
input_df = pd.concat([input_df.drop('Geography', axis= 1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [30]:
# Ensure the input data matches the scaler's expected features
columns_to_scale = scaler.feature_names_in_  # Get the feature names used during fitting
input_df_scaled = input_df[columns_to_scale]  # Select only the relevant columns

# Scaling the input data
input_scaled = scaler.transform(input_df_scaled)

In [31]:
# Prediction 
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step


array([[0.27364862]], dtype=float32)

In [32]:
prediction_prob = prediction[0][0]

In [33]:
if prediction_prob >= 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
